In [1]:
import os
current_dir = os.getcwd()
current_dir

'/Users/1112480/Study/MABs/yahoo'

In [2]:
data_path = '../../data/R6'

In [3]:
import numpy as np
import pandas as pd
import math
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.decomposition import PCA, TruncatedSVD
import networkx as nx

import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")

from scipy import sparse as sp

In [4]:
#function for extracting information from each row of Yahoo data

def extract2(row):
    row=row[0].split(' ')
    if (len(row)-10)%7!=0:
        findex=row.index('7:1.000000')
        row.pop(findex-1)
        row.remove('7:1.000000')
    action_v=int(row[1])
    reward=int(row[2])
    user_feature=[float(row[j][2:]) for j in range(4,10)][:5]
    N=int((len(row)-10)/7)
    articles=[]
    item_features=[]
    for k in range(N):
        articles=articles+[int(row[10+k*7][1:])]
        article_f=[float(row[j][2:]) for j in range(11+k*7,11+(k+1)*7-1)]
        item_features.append(article_f[:5])
    if action_v in articles:
        action_i=articles.index(action_v)
    else:
        action_i='no matching article'
    return({'action_ind':action_i, 'reward':reward, 'user_feature':user_feature, 'item_features':item_features,'arm_num':N})  

In [5]:
# import Yahoo data
df_raw = pd.read_csv(os.path.join(data_path,'ydata-fp-td-clicks-v1_0.20090501.gz'), compression='gzip', header=0, sep=',', quotechar='"')
data_raw = np.array(df_raw)

In [8]:
# extract user information

history_users = np.array([ extract2(data_raw[i])['user_feature'] for i in range(data_raw.shape[0])  ])
print(history_users.shape)


(4681991, 5)


In [9]:
history_users = pd.DataFrame(history_users)

In [10]:
history_users['roundID'] = np.arange(history_users.shape[0])


In [11]:
# extract unique users
df_users = history_users[ [0,1,2,3,4] ].drop_duplicates()

/Users/1112480/opt/miniconda3/envs/ws3_8/lib/python3.8/site-packages/pandas/core/frame.py:6250: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  diff = Index(subset).difference(self.columns)


In [12]:
# make user ID
print(df_users.shape)
df_users['userID'] = np.arange(df_users.shape[0])
print(df_users)

(3284134, 5)
                0         1         2         3         4   userID
0        0.088932  0.003915  0.592269  0.314084  0.000801        0
1        0.000293  0.000002  0.000411  0.000770  0.998524        1
2        0.011889  0.863387  0.000385  0.124338  0.000001        2
3        0.000054  0.000002  0.000022  0.000087  0.999835        3
4        0.000403  0.000009  0.000109  0.000288  0.999191        4
...           ...       ...       ...       ...       ...      ...
4681986  0.000963  0.000001  0.006049  0.000674  0.992312  3284129
4681987  0.025497  0.000146  0.066751  0.038620  0.868985  3284130
4681988  0.103932  0.836199  0.000203  0.059307  0.000359  3284131
4681989  0.327085  0.076663  0.083707  0.510996  0.001549  3284132
4681990  0.058959  0.000292  0.878861  0.061729  0.000159  3284133

[3284134 rows x 6 columns]


In [14]:
pd.DataFrame(history_users).head()

,0,1,2,3,4,roundID
0,0.088932,0.003915,0.592269,0.314084,0.000801,0
1,0.000293,0.000002,0.000411,0.000770,0.998524,1
2,0.011889,0.863387,0.000385,0.124338,0.000001,2
3,0.000054,0.000002,0.000022,0.000087,0.999835,3
4,0.000403,0.000009,0.000109,0.000288,0.999191,4


In [15]:
history_users_with_id = history_users.merge(df_users, on=[0, 1, 2, 3, 4], how="outer")

In [16]:
history_users_with_id = history_users_with_id.sort_values(by=['roundID'])

In [17]:
print(history_users_with_id)

                0         1         2         3         4  roundID   userID
0        0.088932  0.003915  0.592269  0.314084  0.000801        0        0
1        0.000293  0.000002  0.000411  0.000770  0.998524        1        1
2        0.011889  0.863387  0.000385  0.124338  0.000001        2        2
4        0.000054  0.000002  0.000022  0.000087  0.999835        3        3
5        0.000403  0.000009  0.000109  0.000288  0.999191        4        4
...           ...       ...       ...       ...       ...      ...      ...
4681986  0.000963  0.000001  0.006049  0.000674  0.992312  4681986  3284129
4681987  0.025497  0.000146  0.066751  0.038620  0.868985  4681987  3284130
4681988  0.103932  0.836199  0.000203  0.059307  0.000359  4681988  3284131
4681989  0.327085  0.076663  0.083707  0.510996  0.001549  4681989  3284132
4681990  0.058959  0.000292  0.878861  0.061729  0.000159  4681990  3284133

[4681991 rows x 7 columns]


In [18]:
user_freqs = history_users_with_id['userID'].value_counts(normalize=False,sort=True)
user_freqs_IDs = np.array(list(user_freqs.index))
user_freqs = np.array(user_freqs)

In [19]:
user_freqs

array([78683, 47010, 33305, ...,     1,     1,     1])

In [20]:
#extract user IDs which appear more than 50 times

USER_THRESHOLD = 50
USER_NUM = sum( user_freqs >= USER_THRESHOLD )
user_top_IDs = user_freqs_IDs[:USER_NUM]

In [21]:
history_users_with_id_reduced = history_users_with_id.loc[np.isin(history_users_with_id['userID'], user_top_IDs)]
print(history_users_with_id_reduced)
print(history_users_with_id_reduced.shape)
df_users_reduced = df_users.loc[np.isin(df_users['userID'], user_top_IDs)]
metadata_users_reduced = df_users_reduced.merge(pd.DataFrame({'userID':user_freqs_IDs, 'freq':user_freqs}), on='userID', how="inner" )
print(metadata_users_reduced.shape )
print(metadata_users_reduced.head())

               0         1         2         3         4  roundID  userID
69      0.000069  0.000002  0.000009  0.000043  0.999876       20      20
10116   0.010609  0.000910  0.008778  0.055681  0.924022       24      24
10894   0.519539  0.062237  0.058665  0.353326  0.006234       27      27
13215   0.009571  0.000864  0.001399  0.009514  0.978653       28      28
13782   0.000494  0.000015  0.000084  0.000416  0.998991       29      29
...          ...       ...       ...       ...       ...      ...     ...
399860  0.258297  0.001773  0.658313  0.079241  0.002376  4681967     374
457014  0.605156  0.077316  0.029215  0.270011  0.018302  4681977     463
457015  0.605156  0.077316  0.029215  0.270011  0.018302  4681980     463
312998  0.664068  0.073064  0.041332  0.212738  0.008798  4681984     213
174618  0.007116  0.000410  0.001595  0.009283  0.981597  4681985     108

[877969 rows x 7 columns]
(877969, 7)
(835, 7)
          0         1         2         3         4  userID   fr

In [ ]:
real_data_dir = current_dir + "/" + "preproc_output"

if not os.path.exists(real_data_dir):
    os.makedirs(real_data_dir)

In [ ]:
history_users_with_id_reduced.to_csv(real_data_dir+'/history_users_thr'+str(USER_THRESHOLD)+'_1.csv')
metadata_users_reduced.to_csv(real_data_dir+'/metadata_users_thr'+str(USER_THRESHOLD)+'.csv')

In [22]:
# create laplacian matrix based on distance and RBF kernel
rbfmat = rbf_kernel( np.array(metadata_users_reduced)[:,:5] , gamma = 5) 

In [23]:
(rbfmat > 0.03).sum() / metadata_users_reduced.shape[0]**2

0.5032249273907275

In [ ]:
np.save(real_data_dir+"/adjmtx_thr50_sp50percent.npy", (rbfmat > 0.03).astype(int))

In [ ]:
for i in range(2,11):
    if i<10:
        file='ydata-fp-td-clicks-v1_0.2009050'+str(i)+'.gz'
    else:
        file='ydata-fp-td-clicks-v1_0.200905'+str(i)+'.gz'

    df_raw = pd.read_csv(file, compression='gzip', header=0, sep=',', quotechar='"')

    data_raw = np.array(df_raw)

    history_users = np.array([ extract2(data_raw[i])['user_feature'] for i in range(data_raw.shape[0])  ])

    history_users = pd.DataFrame(history_users)

    history_users['roundID'] = np.arange(history_users.shape[0])

    history_users_with_id = history_users.merge(df_users, on=[0, 1, 2, 3, 4], how="outer")

    history_users_with_id = history_users_with_id.sort_values(by=['roundID'])

    history_users_with_id_reduced = history_users_with_id.loc[np.isin(history_users_with_id['userID'], user_top_IDs)]

    history_users_with_id_reduced.to_csv(real_data_dir+'/history_users_thr'+str(USER_THRESHOLD)+'_'+str(i)+'.csv')